In [72]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import layers
from tensorflow import keras

import numpy as np
import tensorflow as tf
import PIL

def getFlatArrayFromImage(path):
  img = PIL.Image.open(path).convert('L')
  arr = np.array(img)

  flat_arr = arr.ravel()
  return flat_arr

path = '/home/andrew/Recognizerka/fonts/png32_rotated.npz'
with np.load(path) as data:
    train_examples = data['x_train']
    train_labels = data['y_train']
    test_examples = data['x_test']
    test_labels = data['y_test']

In [73]:
model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(2, (2, 2), activation='relu', input_shape=(32, 32, 1)),
#     tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(8, (4, 4), activation='relu', input_shape=(32, 32, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(66, activation='softmax')
])


In [74]:
x_train = train_examples

new_y_labels = np.zeros(8910)
for i in range(8910):
    new_y_labels[i], = np.where(train_labels[i] == 1)

y_train = new_y_labels

x_test = test_examples

new_y_labels = np.zeros(1782)
for i in range(1782):
    new_y_labels[i], = np.where(test_labels[i] == 1)

y_test = new_y_labels

In [75]:
# Preprocess the data (these are Numpy arrays)
x_train = x_train.reshape(8910, 32, 32, 1).astype('float32') / 255
x_test = x_test.reshape(1782, 32, 32, 1).astype('float32') / 255

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Reserve 10,000 samples for validation
x_val = x_train[-1782:]
y_val = y_train[-1782:]
x_train = x_train[:-1782]
y_train = y_train[:-1782]

model.compile(optimizer='adam',  # Optimizer
              # Loss function to minimize
              loss=keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              # List of metrics to monitor
              metrics=['sparse_categorical_accuracy'])

In [78]:
print('# Fit model on training data')
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=30,
                    shuffle=True,
                    validation_data=(x_val, y_val))

# Fit model on training data
Train on 7128 samples, validate on 1782 samples
Epoch 1/30
7128/7128 [==============================] - 2s 268us/sample - loss: 3.6132 - sparse_categorical_accuracy: 0.6027 - val_loss: 3.9892 - val_sparse_categorical_accuracy: 0.2301
Epoch 2/30
7128/7128 [==============================] - 2s 238us/sample - loss: 3.6068 - sparse_categorical_accuracy: 0.6076 - val_loss: 3.9796 - val_sparse_categorical_accuracy: 0.2357
Epoch 3/30
7128/7128 [==============================] - 2s 244us/sample - loss: 3.6042 - sparse_categorical_accuracy: 0.6098 - val_loss: 3.9864 - val_sparse_categorical_accuracy: 0.2256
Epoch 4/30
7128/7128 [==============================] - 2s 242us/sample - loss: 3.6040 - sparse_categorical_accuracy: 0.6098 - val_loss: 3.9832 - val_sparse_categorical_accuracy: 0.2312
Epoch 5/30
7128/7128 [==============================] - 2s 242us/sample - loss: 3.6039 - sparse_categorical_accuracy: 0.6098 - val_loss: 3.9794 - val_sparse_categorical_accuracy: 

KeyboardInterrupt: 

In [ ]:
print('\nhistory dict:', history.history)

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_test, y_test, batch_size=64)
print('test loss, test acc:', results)

In [ ]:
# create seporate test set
files = [
    '18847_l_6.png',
    '19046_u_9.png',
    '19051_l_0.png',
    '19081_l_0.png',
    '19081_u_17.png'
]
path_to_files = "/home/andrew/Recognizerka/fonts/png_32_32/"

flen = len(files)
my_test_x = np.zeros( (flen, 32, 32, 1) )
print(my_test_x.shape)
k = 0
for file in files:
    my_test_x[k] = getFlatArrayFromImage(path_to_files+file).reshape(32, 32, 1)
    k+=1

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(my_test_x)
print('predictions shape:', predictions.shape)

In [ ]:
def get_char_from_pred(prediction):
    from draft import RUS_CHARS
    return RUS_CHARS[np.argmax(prediction)].decode('utf-8')

for pred in predictions:
    print(get_char_from_pred(pred))